<a href="https://colab.research.google.com/github/guastiwilsonjr/data_func_res/blob/main/Artigo_ICCSA_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação da bibliotecas necessárias ao funcionamento do algoritmo

In [ ]:
!pip install tensorflow
!pip install numpy
!pip install matplotlib

Atualizar funções para a versão mais recente do Tensorflow

Referências de recursos da biblioteca
https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense

Algoritmo básico - resolução de uma EDO


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
# import tensorflow.compat.v1 as tf

from matplotlib import rc

from tempfile import TemporaryFile

# # FUNCIONALIDADE QUE PERMITE A COMPATIBILIDADE COM A VERSÃO 1.5 DO TENSORFLOW
# tf.disable_v2_behavior()


# Limpa a pilha de grafos padrão e redefine o grafo padrão global
# tf.reset_default_graph()

#Define os valores aleatórios no nível do grafo para o grafo padrão
# tf.set_random_seed(1000)
# ERRADO

#PROBLEMA A SER RESOLVIDO
#EQUACOES DIFERENCIAIS ORDINARIAS DE PRIMEIRA ORDEM (PROBLEMA DE VALOR INICIAL - PVI)
#====================================================================================
#FORMA DO PVI
#  du(t)/dt + p(t)u(t) = q(t),   t \in [t_0, t_f] 
#               u(t_0) = u_0
#
#Esse PVI pode ser escrito da forma du(t)/dt = f(t,u)
#onde f(t,u) = q(t) - p(t)u(t)
#Nesse problema, t_0 = 0, t_f =1, u_0 = 1 e p e q sao descritas pelas funcoes p e q
#===================================================================================


#===================================================================================
#FUNCAO QUE DESCREVE A FUNCAO p(t)
def p(x):
    '''
        Left part of initial equation
    '''
    return x + (1. + 3.*x**2) / (1. + x + x**3)
#===================================================================================

#===================================================================================
#FUNCAO QUE DESCREVE A FUNCAO q(t)
def q(x):
    '''
        Right part of initial equation
    '''
    return x**3 + 2.*x + x**2 * ((1. + 3.*x**2) / (1. + x + x**3))
#===================================================================================

#===================================================================================
#SOLUCAO EXATA
def g_analytic(x):
    # return x*(1-x)*np.exp(x)
    return ((np.exp((-x**2)/2.)) / (1. + x + x**3)) + x**2
#===================================================================================

Nx = 10

# RETORNAM NÚMEROS IGUALMENTE ESPAÇADOS DENTRO DO INTERVALO ESPECIFICADO [NX]
x = np.linspace(0,1, Nx)


# CONVERTER OBJETO PYTHON PARA OBJETOS DO TIPO TENSOR
x_tf = tf.convert_to_tensor(x.reshape(-1,1),dtype=tf.float64)
g1 = tf.constant(1.,dtype=tf.float64)
g2 = tf.constant(2.,dtype=tf.float64)
g3 = tf.constant(3.,dtype=tf.float64)


# NÚMERO DE ITERAÇÕES
num_iter = 10000

# VARIÁVEL PARA O CÁLCULO DO DECAIMENTO DO FUNCIONAL RESÍDUO
vector_functional_dominio = np.zeros(num_iter)
vector_functional = np.zeros(num_iter)

# NÚMERO DE NEURÔNIOS EM CADA CAMADA, SÃO DUAS CAMADAS
num_hidden_neurons = [10,10]

# NÚMERO DE CAMADAS É DEFINIDO PELA QUANTIDADE DE ELEMENTOS NA LISTA
num_hidden_layers = np.size(num_hidden_neurons)


# CONSTRUÇÃO DA REDE
# AGRUPAMENTO DE CADA PASSO NA CONSTRUÇÃO DA REDE NEURAL
with tf.name_scope('dnn'):

    # CAMADA DE ENTRADA
    previous_layer = x_tf

    # CAMADAS ESCONDIDAS
    for l in range(num_hidden_layers):
        # A VERSÃO MAIS ATUAL DA FUNÇÃO UTILIZA A INTERFACE DO KERAS 
        #current_layer = tf2.keras.layers.Dense()

        # tf.layers.dense --> FUNÇÃO DESCONTINUADA
        # DENSE REPRESENTA A CAMADA: SAÍDA = FUNÇÃO_ATIVAÇÃO (ENTRADA * KERNEL + BIAS), ONDE
        # KERNEL É A MATRIZ DE PESOS E O BIAS É UM VETOR CRIADO PELA CAMADA
        #(ENTRADA, NÚMERO DE DIMENSÃO DA SAÍDA, NOME, FUNÇÃO DE ATIVAÇÃO, INICIALIZAÇÃO DOS PESOS DA MATRIZ)
        # current_layer = tf.layers.dense(previous_layer, num_hidden_neurons[l], name='hidden%d'%(l+1), activation=tf.nn.sigmoid)
        current_layer = tf.keras.layers.Dense(previous_layer, num_hidden_neurons[l], name='hidden%d'%(l+1), activation=tf.nn.relu,kernel_initializer='glorot_uniform')
        previous_layer = current_layer

    # CAMADA DE SAÍDA
    dnn_output = tf.layers.dense(previous_layer, 1, name='output')


# DEFINIÇÃO DA FUNÇÃO TRIAL E DA SEÇÃO CUSTO
with tf.name_scope('custo'):
    # FUNÇÃO TRIAL
    # g_trial = x_tf * dnn_output
    g_trial = 1.  + x_tf * dnn_output
    # g_trial = x_tf*(x_tf-1)*dnn_output
    # psy_t = 1. + xi * net_out

    # DERIVADA DA FUNÇÃO TRIAL
    d_g_trial = tf.gradients(g_trial,x_tf)
    
    # DERIVADA SEGUNDA DA FUNÇÃO TRIAL
    # d2_g_trial = tf.gradients(d_g_trial,x_tf)

    # right_side = (3*x_tf + x_tf**2)*tf.exp(x_tf)
    # right_side = x_tf**3 + 2*x_tf + x_tf**2 * ((1 + 3*x_tf**2) / (1 + x_tf + x_tf**3))
    right_side = q(x_tf) - g_trial*p(x_tf)
    # right_side = q(g_trial) - x_tf*p(g_trial)

    err = tf.square( d_g_trial[0] - right_side)
    custo = tf.reduce_sum(err, name = 'custo')

# ----------------------------------------------------
# ESCOLHA DO MÉTODO PARA MINIMIZAR A FUNÇÃO CUSTO
# ----------------------------------------------------

# TAXA DE APRENDIZAGEM
learning_rate = 0.001
with tf.name_scope('treinamento'):
    # MÉTODO DE MINIMIZAÇÃO - GRADIENT DESCENT
    # optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    
    # MÉTODO DE MINIMIZAÇÃO - ADAM
    optimizer = tf.train.AdamOptimizer(learning_rate)

    # MÉTODO DE MINIMIZAÇÃO - ADADELTA
    # optimizer = tf.train.AdadeltaOptimizer(learning_rate)

    # MÉTODO DE MINIMIZAÇÃO - ADADELTA
    # optimizer = tf.train.AdadeltaOptimizer(learning_rate)


    # optimizer = tfp.optimizer.StochasticGradientLangevinDynamics(learning_rate)
    # optimizer = tfp.optimizer.bfgs_minimize(quadratic_loss_and_gradient, initial_position=start, 
    #     tolerance=1e-8)

    traning_op = optimizer.minimize(custo)

g_dnn_tf = None


# DEFINE O NÓ QUE INICIALIZA TODOS OS OUTROS NÓS DO GRAFO - NO CASO, AS CAMADAS 
init = tf.global_variables_initializer()

# print("valor de init:", init)



# ETAPA DE CONSTRUÇÃO DO GRAFO - REDE
with tf.Session() as sess:
   
    # INICIALIZA TODA A REDE
    # EM TODAS AS ETAPAS SERÃO CHAMADAS AS SESSÕES CONSTRUIDAS ACIMA
    init.run()

    
    # AVALIAR O CUSTO INICIAL - CHAMA A SESSÃO CUSTO
    # print('Custo inicial: %g'%custo.eval())


    # INICIO DO TREINAMENTO DA REDE - CHAMA A SESSÃO TREINAMENTO
    for i in range(num_iter):
        sess.run(traning_op)
        vector_functional_dominio[i] = i;
        vector_functional[i] = custo.eval()


 


    # CÁLCULO DO VALOR MÍNIMO DO FUNCIONAL
    min_Jr = np.min(err.eval())
    

    # ARMAZENAMENTO DO RESULTADO FINAL
    # EVAL -> EXECUTA/TRANSFORMA O CONTEÚDO COMO/EM UMA EXPRESSÃO PYTHON
    g_dnn_tf = g_trial.eval()

    
    # # FAZ A IMPRESSÃO DO RESULTADO FINAL DO GRAPO
    # writer = tf.summary.FileWriter("./output", sess.graph)
    # writer.close()

g_analytical = g_analytic(x)

diff_tf = g_dnn_tf - g_analytical.reshape(-1,1)
#=============================================================================
#                   RUNGE KUTTA DE 2ª ORDEM
#=============================================================================

def feval(funcName, *args):
    return eval(funcName)(*args)


def RK2B(func, yinit, x_range, h):
    m = len(yinit)
    n = int((x_range[-1] - x_range[0])/h)
    
    x = x_range[0]
    y = yinit
    
    # Containers for solutions
    xsol = np.empty(0)
    xsol = np.append(xsol, x)

    ysol = np.empty(0)
    ysol = np.append(ysol, y)

    for i in range(n):
        k1 = feval(func, x, y)

        ypredictor = y + k1 * (h/2)

        k2 = feval(func, x+h/2, ypredictor)

        for j in range(m):
            y[j] = y[j] + h*k2[j]

        x = x + h
        xsol = np.append(xsol, x)

        for r in range(len(y)):
            ysol = np.append(ysol, y[r])  

    return [xsol, ysol]


def myFunc(x, y):
    dy = np.zeros((len(y)))
    # dy[0] = np.exp(-2 * x) - 2 * y[0]
    # dy[0] = x / (2*x**2+1)**0.5
    dy[0] = q(x) - y[0] * p(x)
    return dy

# -----------------------
h = 0.1
# x = np.array([0, 1])

# nx = 20

x = np.linspace(0, 1, Nx)

# mudança do valor inicial
# yinit = np.array([1.0/10])
yinit = np.array([1.0])



# print('Resultado de RK2', [ts, ys])

#=====METODO DE DIFERENCAS FINITAS EULER EXPLICITO ==================
#Inicialização com os parâmetros iniciais
#fd = finite difference
y_space = g_analytic(x)
dx = 1. / (Nx-1)
psy_fd = np.zeros_like(y_space) #np.zeros_like retorna um vetor nulo
psy_fd[0] = 1. # Condição inicial

[ts, ys] = RK2B('myFunc', yinit, x, dx)

for i in range(1, len(x)):
    psy_fd[i] = psy_fd[i-1] + q(x[i]) * dx - psy_fd[i-1] * p(x[i]) * dx





# IMPORTANTE  - CÁLCULO DAS MÉTRICAS
# CÁLCULO DA NORMA DO INFINITO
# print('\nNorma do Infinito: %g'%np.max(np.abs(diff_tf)))
print('\nNorma do Infinito: {:.3e}'.format(np.max(np.abs(diff_tf))))


# # # CÁLCULO DA NORMA L2
norma_l2 = np.sqrt(np.cumsum(np.power(diff_tf,2))[Nx-1])
# # print('\nNorma L2: {:.7f}'.format(norma_l2))
print('\nNorma L2: {:.3e}'.format(norma_l2))

# # # IMPRESSÃO DO FUNCIONAL
# # print('\nValor mínimo do funcional Jr: {:.7f}'.format(min_Jr))
print('\nValor mínimo do funcional Jr: {:.3e}'.format(min_Jr))

# =========================================================
#
# COMPARAÇÃO ENTRE OS MÉTODOS RK2 ORDEM E DIFERENÇAS FINITAS
#
# =========================================================
# plt.figure()
# plt.plot(x, g_analytical, linestyle='solid',color='blue', marker="x", lw=2, label='Solução exata')
# plt.ylabel(r'Microstrain [$\mu \epsilon$]')
# plt.plot(ts, ys, linestyle='solid',color='green', marker="^", label='Runge-Kutta 2ª ordem')
# plt.plot(x, g_dnn_tf, linestyle='solid', color='red', marker="s", label='Redes Neurais')
# plt.plot(x, psy_fd, linestyle='solid', color='black', marker="o", label='Euler explícito')
# plt.grid(True)
# plt.title('du(x)/dx + p(x) u(x) = q(x)')
# plt.legend()
# plt.show()


#================================================================================================================================
#                                                    ERROS - NORMA INFINITO
#================================================================================================================================

def norma_infinito(u, v, n):
    maior = abs(u[0]-v[0])
    for i in range(n):
        valor = abs(u[i]-v[i])
        if (valor > maior):
            maior = valor
    return maior

# print('Error - infinity norm - forward Euler method = ','%.3e' % norma_infinito(g_analytical, psy_fd, Nx))
# print('Error - infinity norm - RK2 method = ','%.3e' % norma_infinito(g_analytical, ys, Nx))
# print('Error - infinity norm - ANN method = ','%.3e' % norma_infinito(g_analytical, g_dnn_tf, Nx))


# print('\nNorma do Infinito: %g'%np.max(np.abs(diff_tf)))

# # Plot the result
# plt.figure()

# # plt.title('du(t)/dt + p(t) u(t) = q(t)')

# plt.plot(x, g_dnn_tf, linestyle='solid', color='red', marker="s", label='Solução via Redes Neurais')
# plt.plot(x, g_analytical, linestyle='solid', color='blue', marker="x", label='Solução exata')
# plt.xlabel('t')
# plt.ylabel('y')
# plt.grid(True)
# plt.legend()
# plt.show()


# plt.yscale('log',basey=2) 

# outfile = TemporaryFile()
# np.save(outfile, vector_functional)

# _ = outfile.seek(0) # Apenas para simular a abertura e fechamento do arquivo
# teste = np.load(outfile)
# print(teste)


# with open('teste/relu.npy', 'wb') as f:
#     np.save(f, vector_functional)
# with open('teste/relu.npy', 'rb') as f:
#     a = np.load(f)

# print(a)
print('dominio')
print(vector_functional_dominio)

# SALVAR OS PONTOS DO DOMÍNIO
# with open('vector_functional_dominio.npy', 'wb') as g:
#     np.save(g, vector_functional_dominio)
# with open('vector_functional_dominio.npy', 'rb') as g:
#     b= np.load(g) 
# print('dominio_salvo')
# print(b)
# DEACAIMENTO DO FUNCIONAL RESÍDUO
plt.plot(b, vector_functional, color='blue', label='Funcional Resíduo')
plt.rc('text', usetex=True)
plt.xlabel('Nº iterações')
plt.ylabel('$J_{R}$')
plt.grid(True)
plt.legend()
plt.semilogy()
plt.show()



TypeError: ignored